In [1]:
#note that the adjacency matrix for a graph is a 0/1 matrix
#0/1 exists in any field, so we can view A \in (F_p)^{n^2} \cong F_{p^{n^2}} = F_q
#thus every adjacency matrix A can be viewed as an element of F_q

In [2]:
#determine if A is a 0/1 adjacency matrix with zeros on diagonal, symmetric
from sage.misc.flatten import flatten
def is_adj_matrix(A):
    #deal with case A is list of lists or matrix
    if type(A) == list:
        flat_A = flatten(A) #make the adjacency matrix into a list
        n = len(A) #get the number of columns of A
    else:
        flat_A = flatten([list(row) for row in A.rows()])
        n = A.ncols()
    zero_one = all(flat_A[k] == 0 or flat_A[k] == 1 for k in range(n^2))
    symmetric = matrix(A) == matrix(A).transpose()
    zero_diagonal = all(A[k][k] == 0 for k in range(n))
    return zero_one and symmetric and zero_diagonal

In [3]:
#given an adjacency matrix A and a prime p
#embed the graph A into F_q where q=p^{n^2} and n is the number of vertices
def graph_embedding(A,p):
    try:
        assert p in Primes() #make sure p is prime
    except AssertionError:
        print("p must be prime")
        return
    if type(A) == list:
        n = len(A) #get the number of columns of A
        flat_A = flatten(A) #make the adjacency matrix into a list
    else:
        n = A.ncols()
        flat_A = flatten([list(row) for row in A.rows()])
    q = p**(n**2) #compute q=p^{n^2}
    alpha = GF(q).gen() #get a primitive element for the finite field F_q
    try:
        assert is_adj_matrix(A) #ensure A is an adjacency matrix
    except AssertionError:
        print("A must be a 0/1 adjacency matrix")
        return
    return sum([flat_A[k]*alpha**k for k in range(n^2)]) #let {1,alpha,...,alpha^{n^2-1}} be a basis and map the matrix into F_q

In [4]:
#naively find k such that alpha^k = x 
#where alpha is a generator for the finite field
def gen_power(x):
    alpha = x.parent().gen()
    k = 0
    while k < x.parent().order():
        if alpha**k == x:
            return k
        else:
            k += 1

In [32]:
#compute all adjacency matrices
def all_adj_mats(n):
    all_zero_one_mats = [matrix(ZZ,A) for A in MatrixSpace(GF(2),n,n)] #compute all 0/1 matrices
    return [A for A in all_zero_one_mats if is_adj_matrix(A)]

In [6]:
#compute integer corresponding to adjacency matrix
def graph_int(A,p):
    return gen_power(graph_embedding(A,p))

In [7]:
#assign a unique integer to the pair (i,j) as an element of a matrix
#count number of elements to right of diagonal until you reach (i,j)
def unique_int_pair(n,i,j):
    return sum(n-k for k in range(1,i)) + (j-i)

In [8]:
#compute the integer corresponding to a labeled graph given as an adjacency matrix A
def graph_int_basic(A):
    try:
        assert is_adj_matrix(A)
    except AssertionError:
        print("A must be 0/1 adjacency matrix")
    if type(A) == list:
        flat_A = flatten(A) #make the adjacency matrix into a list
        n = len(A) #get the number of columns of A
    else:
        n = A.ncols()
        flat_A = flatten([list(row) for row in A.rows()])
    return sum(flatten([[A[i][j]*(2**unique_int_pair(n,i+1,j+1)) for j in range(i+1,n)] for i in range(n)]))

In [83]:
#compute the basic graph integer k and a generator for F_q alpha and compute alpha^k
def graph_embedding_basic(A,p):
    try:
        assert is_adj_matrix(A)
    except AssertionError:
        print("A must be 0/1 adjacency matrix")
    if type(A) == list:
        flat_A = flatten(A) #make the adjacency matrix into a list
        n = len(A) #get the number of columns of A
    else:
        n = A.ncols()
        flat_A = flatten([list(row) for row in A.rows()])
    #ensure the cyclic group of size q-1 is at least as big as the number of graphs, 2^((n choose 2)+1)
    r = 1
    while p**r - 1 < 2**(binomial(n,2)+1):
        r += 1
    q = p**r
    k = graph_int_basic(A)
    alpha = GF(q,modulus="primitive").gen() #find a primitive generator of the multiplicative group of F_q
    return alpha**k

In [20]:
#define the action of the symmetric group on graphs
#sigma is an element of the symmetric group, A is an adjacency matrix
#returns sigma*A
def sym_act(sigma,A):
    try:
        assert is_adj_matrix(A)
    except AssertionError:
        print("A must be 0/1 adjacency matrix")
    if type(A) == list:
        A = matrix(A) #get the number of columns of A
    n = A.ncols()
    try:
        assert sigma in SymmetricGroup(n)
    except AssertionError():
        print("sigma must be an element of the symmtric group")
    return sigma.matrix()*A*sigma.matrix().transpose()

In [42]:
#define function which hides the automorphism group of the adjacency matrix A
#the output is an adjacency matrix embedded into a finite field
def f_int(A,sigma):
    return graph_int_basic(sym_act(sigma,A))

In [47]:
#return the graph embedding of sigma*A
def f_embed(A,sigma,p):
    return graph_embedding_basic(sym_act(sigma,A),p)

In [95]:
#compute the Fourier coefficients of f: S_n --> F_q using the SGA DFT
def fourier_coefficients(A,p):
    if type(A) == list:
        A = matrix(A)
    n = A.ncols()
    sga_dft = SymmetricGroup(n).algebra(GF(p)).dft()
    f_list = [f_embed(A,sigma,p) for sigma in SymmetricGroup(n)]
    return sga_dft*vector(f_list)

In [84]:
A = all_adj_mats(3)[4]; A

[0 1 1]
[1 0 0]
[1 0 0]

In [85]:
#compute the value of f for the entire symmetric group
[f_int(A,sigma) for sigma in SymmetricGroup(3)]

[6, 10, 12, 6, 12, 10]

In [86]:
graph_embedding_basic(A,23)

8

In [87]:
[f_embed(A,sigma,23) for sigma in SymmetricGroup(3)]

[8, 9, 18, 8, 18, 9]

In [55]:
SymmetricGroup(3).algebra(GF(5)).dft()

[1 1 1 1 1 1]
[1 2 2 3 3 4]
[0 3 2 2 3 0]
[0 1 4 1 4 0]
[1 2 2 2 2 1]
[1 1 1 4 4 4]

In [96]:
fourier_coefficients(A,23)

(1, 10, 5, 4, 2, 0)